# Generalize tomography scans

Test implementation of a [generalized tomography class](https://github.com/APS-2BM-MIC/ipython-user2bmb/issues/49#issuecomment-438788678) for use at the APS.

In [1]:
from collections import OrderedDict
import datetime
import numpy
import os
import time

from ophyd import Device, Component, Signal
from ophyd import EpicsSignal, EpicsMotor
from ophyd import AreaDetector, ADComponent, SimDetectorCam
from ophyd import SingleTrigger, ImagePlugin, HDF5Plugin

import bluesky.plans as bp
import bluesky.plan_stubs as bps
import bluesky.preprocessors as bpp
from bluesky.simulators import summarize_plan
from bluesky.utils import ts_msg_hook
from bluesky import RunEngine

import APS_BlueSky_tools.devices as APS_devices
import APS_BlueSky_tools.plans as APS_plans

prepare the area detector class

In [2]:
class MyAreaDetector(SingleTrigger, AreaDetector):
    """AD detector as expected for tomography"""
    
    cam = ADComponent(SimDetectorCam, "cam1:")
    image = ADComponent(ImagePlugin, "image1:")
    hdf1 = ADComponent(
        HDF5Plugin, 
        suffix="HDF1:",
        #root=HDF5_FILE_PATH_ROOT,               # for databroker
        # write_path_template=HDF5_FILE_PATH,     # for EPICS AD
)

define a base class

In [3]:
class TomographyScanBase(object):
    """
    generalization of a (repeated) tomography scan
    
    ::
    
        tomo_plan()
            iterate
                before_tomo_sequence
                measure_darks
                measure_flats
                before_tomo_scan
                tomo_scan
                after_tomo_scan
                after_tomo_sequence

    """
    # TODO: acquire time
    # TODO: darks, flats, images all in one HDF5 file
    # TODO: configure AD for HDF5 and databroker
    
    def __init__(self, image_detector, rotation_axis, translation_axis, shutter=None, iterations=1, in_pos=None, out_pos=None, **kwargs):
        # be very defensive here, can eliminate long exception traces at run time
        assert isinstance(image_detector, AreaDetector)
        assert isinstance(rotation_axis, EpicsMotor)
        assert isinstance(translation_axis, EpicsMotor)
        assert isinstance(iterations, int)
        assert iterations > 0
        assert isinstance(shutter, (type(None), Device))   # TODO: could be more specific than "Device"
        assert in_pos is not None, "in_pos must be defined as a valid position for translation_axis"
        assert out_pos is not None, "out_pos must be defined as a valid position for translation_axis"

        self.detector = image_detector        # ophyd AreaDetector Device object
        self.rotator = rotation_axis           # ophyd EpicsMotor Device object
        self.translator = translation_axis    # ophyd EpicsMotor Device object
        self.iterations = max(1, iterations)  # positive integer (or ? should report as OutOfRangeError ?)
        self.shutter = shutter                # ophyd Device object or ``None``
        
        # float: self.translator == this value ...
        self.in_beam_position = in_pos        # ... when sample IS in beam
        self.out_of_beam_position = out_pos   # ... when sample IS NOT in beam
        
        # number of images to collect
        self.number_of_darks = 3
        self.number_of_flats = 3
        self.number_of_projections = 1800
        self.angle_start = 0
        self.angle_end = 180
        
        self.__tomo_scan_counter = 0          # internal use, reporting

    def before_tomo_sequence(self, *args, **kwargs):
        """
        bluesky plan to initialize for a set of tomo scans
        
        override in subclass as needed for specific instrument
        """
        yield from bps.checkpoint()

    def after_tomo_sequence(self, *args, **kwargs):
        """
        bluesky plan to initialize for a set of tomo scans
        
        override in subclass as needed for specific instrument
        """
        yield from bps.checkpoint()

    def measure_darks(self, *args, **kwargs):
        """
        bluesky plan to measure the dark-field images (shutter closed)
        
        override in subclass as needed for specific instrument
        """
        assert self.number_of_darks >= 0, "number_of_darks must be a non-negative integer"
        if self.number_of_darks == 0:
            yield from bps.checkpoint()   # have to yield some Msg
            print("no darks")
        else:
            t0 = time.time()
            if self.shutter is not None:
                yield from bps.mv(self.shutter, "close")

            for _i in range(self.number_of_darks):
                yield from bps.create(name="darks")
                yield from bps.read(self.detector)
                yield from bps.save()
            msg = "{}: done with {} darks: total time: {} s"
            print(msg.format(
                datetime.datetime.now(), 
                self.number_of_darks,
                time.time()-t0
            ))

    def measure_flats(self, *args, **kwargs):
        """
        bluesky plan to measure the white-field images (no sample in beam, shutter open)
        
        override in subclass as needed for specific instrument
        """
        assert self.number_of_flats >= 0, "number_of_flats must be a non-negative integer"
        if self.number_of_flats == 0:
            yield from bps.checkpoint()   # have to yield some Msg
            print("no flats")
        else:
            assert self.in_beam_position is not None, "must be a valid position"
            assert self.out_of_beam_position is not None, "must be a valid position"

            t0 = time.time()
            if self.shutter is not None:
                yield from bps.abs_set(self.shutter, "open", group="measure_flats_prep")
            yield from bps.abs_set(self.translator, self.out_of_beam_position, group="measure_flats_prep")
            yield from bps.wait(group="measure_flats_prep")

            for _i in range(self.number_of_flats):
                yield from bps.create(name="flats")
                yield from bps.read(self.detector)
                yield from bps.save()

            yield from bps.mv(self.translator, self.in_beam_position)
            msg = "{}: done with {} flats: total time: {} s"
            print(msg.format(
                datetime.datetime.now(), 
                self.number_of_flats,
                time.time()-t0
            ))

    def before_tomo_scan(self, *args, **kwargs):
        """
        bluesky plan to prepare the actual tomography scan
        
        override in subclass as needed for specific instrument
        """
        yield from bps.checkpoint()

        if self.shutter is not None and self.shutter.isClosed:
            yield from bps.abs_set(self.shutter, "open", group="before_tomo_scan")

        yield from bps.abs_set(self.translator, self.in_beam_position, group="before_tomo_scan")
        yield from bps.wait(group="before_tomo_scan")

    def after_tomo_scan(self, *args, **kwargs):
        """
        bluesky plan to finish up after the actual tomography scan
        
        override in subclass as needed for specific instrument
        """
        yield from bps.checkpoint()

        if self.shutter is not None and self.shutter.isOpen:
            yield from bps.abs_set(self.shutter, "close", group="after_tomo_scan")

        yield from bps.abs_set(self.rotator, 0.0, group="after_tomo_scan")
        yield from bps.wait(group="after_tomo_scan")

    def tomo_scan(self, *args, md=None, **kwargs):
        """
        bluesky plan to perform the actual tomography scan
        
        MUST override in subclass
        """
        raise NotImplementedError("must define tomo_scan() method in subclass")

    def cleanup(self):   # must have no arguments if called from bpp.finalize_decorator()
        """
        remove any setup such as monitors, EPICS PV values, motors to default positions, ...
        
        override in subclass as needed for specific instrument
        """
        yield from bps.checkpoint()

    def tomo_plan(self, *args, md=None, **kwargs):
        """
        bluesky plan to run iterated, identical tomography scans
        """
        self.iterations = max(1, self.iterations)   # insurance against improper setting by user
        
        _md = md or OrderedDict()
        _md["tomo_plan"] = "tomo_scan"
        
        self.__tomo_scan_counter = 0
        
        @bpp.stage_decorator([self.detector, self.rotator, self.translator])
        @bpp.run_decorator(md=_md)
        @bpp.finalize_decorator(self.cleanup)
        def tomo_core():   # must have no arguments if called from bps.repeat()
            self.__tomo_scan_counter += 1
            t0 = time.time()
            yield from bps.checkpoint()
            yield from self.before_tomo_sequence()
            yield from self.measure_darks()
            yield from self.measure_flats()
            yield from self.before_tomo_scan()
            yield from self.tomo_scan()
            yield from self.after_tomo_scan()
            yield from self.after_tomo_sequence()

            msg = "{}: iteration {} of {}: total time for iteration: {} s"
            print(msg.format(
                datetime.datetime.now(), 
                self.__tomo_scan_counter, 
                self.iterations,
                time.time()-t0
            ))
        
        yield from bps.repeat(tomo_core, num=self.iterations)

build our own subclass (it will step scan)

In [4]:
class MyTomoScan(TomographyScanBase):
    def tomo_scan(self, *args, md=None, **kwargs):
        yield from bps.checkpoint()
        positions = numpy.linspace(
            self.angle_start,           # could be less or greater than angle_end
            self.angle_end,             # do not reach this value
            self.number_of_projections, 
            endpoint=False,             # do not include end point
            )
        readables = [self.detector, self.rotator, self.translator]
        
        for pos in positions:
            yield from bps.mv(self.rotator, pos)
            yield from bps.trigger_and_read(readables)

prepare a Bluesky RunEngine (no databroker backend now) and add debugging

In [5]:
RE = RunEngine({})
RE.msg_hook = ts_msg_hook

create our virtual tomo instrument

In [6]:
IOC = "gov:"
HDF5_FILE_PATH_ROOT =  "/"
HDF5_FILE_PATH = os.path.join(HDF5_FILE_PATH_ROOT, "tmp", "simdet", "%Y/%m/%d") + "/"

theta = EpicsMotor(IOC+"m1", name="theta")
sample = EpicsMotor(IOC+"m2", name="sample")
shutter = APS_devices.SimulatedApsPssShutterWithStatus(name="shutter")
simdet = MyAreaDetector("otzSIM1:", name="simdet")

create our virtual Tomo Scan object

In [7]:
my_tomo = MyTomoScan(simdet, theta, sample, shutter, in_pos=0, out_pos=-1.5)

cut back on the defaults during testing

In [8]:
my_tomo.angle_start = 0
my_tomo.angle_end = 5
my_tomo.number_of_projections = 10

test the plan for compliance

In [9]:
summarize_plan(my_tomo.tomo_plan())

=================================== Open Run ===================================
shutter -> close
  Read ['simdet']
  Read ['simdet']
  Read ['simdet']
2018-11-15 16:18:42.627043: done with 3 darks: total time: 0.0006270408630371094 s
shutter -> open
sample -> -1.5
  Read ['simdet']
  Read ['simdet']
  Read ['simdet']
sample -> 0
2018-11-15 16:18:42.627382: done with 3 flats: total time: 0.00029397010803222656 s
shutter -> open
sample -> 0
theta -> 0.0
  Read ['simdet', 'theta', 'sample']
theta -> 0.5
  Read ['simdet', 'theta', 'sample']
theta -> 1.0
  Read ['simdet', 'theta', 'sample']
theta -> 1.5
  Read ['simdet', 'theta', 'sample']
theta -> 2.0
  Read ['simdet', 'theta', 'sample']
theta -> 2.5
  Read ['simdet', 'theta', 'sample']
theta -> 3.0
  Read ['simdet', 'theta', 'sample']
theta -> 3.5
  Read ['simdet', 'theta', 'sample']
theta -> 4.0
  Read ['simdet', 'theta', 'sample']
theta -> 4.5
  Read ['simdet', 'theta', 'sample']
theta -> 0.0
2018-11-15 16:18:42.633644: iteration 1 of 

Run the tomography plan

In [10]:
RE(my_tomo.tomo_plan())

16:18:42.647804 checkpoint        -> None            args: (), kwargs: {}
16:18:42.648016 stage             -> simdet          args: (), kwargs: {}
16:18:42.984331 stage             -> theta           args: (), kwargs: {}
16:18:42.984532 stage             -> sample          args: (), kwargs: {}
16:18:42.984633 open_run          -> None            args: (), kwargs: {'tomo_plan': 'tomo_scan'}
16:18:42.987499 checkpoint        -> None            args: (), kwargs: {}
16:18:42.987650 checkpoint        -> None            args: (), kwargs: {}
16:18:42.987949 set               -> shutter         args: ('close',), kwargs: {'group': 'e12c2f9d-2054-4652-868b-759b1414b110'}
16:18:42.988895 wait              -> None            args: (), kwargs: {'group': 'e12c2f9d-2054-4652-868b-759b1414b110'}
16:18:42.989216 create            -> None            args: (), kwargs: {'name': 'darks'}
16:18:42.989298 read              -> simdet          args: (), kwargs: {}
16:18:43.153662 save              -> None    

16:18:49.217847 create            -> None            args: (), kwargs: {'name': 'primary'}
16:18:49.218090 read              -> simdet          args: (), kwargs: {}
16:18:49.218332 read              -> theta           args: (), kwargs: {}
16:18:49.220185 read              -> sample          args: (), kwargs: {}
16:18:49.221299 save              -> None            args: (), kwargs: {}
16:18:49.223310 set               -> theta           args: (3.0,), kwargs: {'group': '38bf9ade-eda5-498e-bc08-9aa4f16dc4e9'}
16:18:49.225560 wait              -> None            args: (), kwargs: {'group': '38bf9ade-eda5-498e-bc08-9aa4f16dc4e9'}
16:18:49.407439 trigger           -> simdet          args: (), kwargs: {'group': 'trigger-cb75ce'}
16:18:49.409502 trigger           -> theta           args: (), kwargs: {'group': 'trigger-cb75ce'}
16:18:49.409905 trigger           -> sample          args: (), kwargs: {'group': 'trigger-cb75ce'}
16:18:49.410610 wait              -> None            args: (), kwargs:

('79ce5de1-c9da-47cb-9717-b48989f529dc',)

Run again, with diagnostics off

In [11]:
RE.msg_hook = None
RE(my_tomo.tomo_plan())

2018-11-15 16:18:51.687663: done with 3 darks: total time: 0.01966261863708496 s
2018-11-15 16:18:55.515773: done with 3 flats: total time: 3.8280248641967773 s
2018-11-15 16:18:58.919763: iteration 1 of 1: total time for iteration: 7.251878023147583 s


('85498016-f273-48db-8d21-54a63090f60f',)